이번 Lab에서는 다층 퍼셉트론과 ReLU 함수, Dropout, Xavier Initialization를 이용해 MNIST 데이터를 분류해보겠습니다.

In [ ]:
# Lab 10 MNIST and softmax
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

필요한 파라미터들을 세팅해줍니다.

In [ ]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
drop_prob = 0.3

torchvision.datasets.dsets.MNIST를 사용하여 MNIST 데이터셋을 불러옵니다.

In [ ]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train = True, #True는 train 데이터
                          transform = transforms.ToTensor(), #데이터를 PyTorch 텐서로 변환
                          download = True) #해당 경로에 MNIST 데이터가 없다면 다운로드

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train = False,  #False는 test 데이터
                         transform = transform.ToTensor(), #데이터를 PyTorch 텐서로 변환
                         download = True) #해당 경로에 MNIST 데이터가 없다면 다운로드

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



이렇게 데이터를 다운로드했다면 데이터로더(DataLoader)를 사용합니다.

In [ ]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, # mnist_train 데이터를 로드할 대상으로 정해줍니다.
                                          batch_size=batch_size, # 위에서 정의한 배치크기를 사용합니다.
                                          shuffle=True, # Epoch마다 데이터 셋을 섞어서 데이터가 학습되는 순서를 바꾸겠습니다.
                                          drop_last=True) #마지막 배치를 버리겠습니다.

이제 신경망을 구현해보겠습니다.

In [ ]:
# nn layers
# MNIST data image of shape 28 * 28 = 784, 0~9까지 10개의 class
linear1 = nn.Linear(784, 512, bias=True) # input layer의 크기를 생각해보세요.
linear2 = nn.Linear(784, 512, bias=True)
linear3 = nn.Linear(784, 512, bias=True)
linear4 = nn.Linear(784, 512, bias=True)
linear5 = nn.Linear(784, 10, bias=True) # output layer의 크기를 생각해보세요.
relu = nn.ReLU() # ReLU를 사용해보겠습니다.
dropout = nn.Dropout(p=drop_prob) # Dropout을 사용해보겠습니다.

In [ ]:
# 가중치 초기화중 xavier 초기화를 사용해보겠습니다.
nn.init.xavier_uniform_(linear1.weight)
nn.init.xavier_uniform_(linear2.weight)
nn.init.xavier_uniform_(linear3.weight)
nn.init.xavier_uniform_(linear4.weight)
nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0615,  0.0453,  0.0754,  ...,  0.1046,  0.0855,  0.0374],
        [ 0.0771,  0.0510,  0.0434,  ..., -0.0905, -0.0800, -0.0707],
        [ 0.0156, -0.0398,  0.0699,  ..., -0.0891, -0.0685,  0.0879],
        ...,
        [-0.0814,  0.0486, -0.1052,  ..., -0.0827, -0.0280, -0.0003],
        [ 0.0770, -0.0791, -0.0681,  ...,  0.0142, -0.0214,  0.0162],
        [ 0.0888, -0.0451, -0.0634,  ..., -0.0700,  0.0365,  0.0334]],
       requires_grad=True)

In [ ]:
# nn.Sequential을 통해 각 층, 렐루 함수, 드롭아웃을 연결해줍니다.
model = nn.Sequential(linear1, relu, dropout,
                            linear2, relu, dropout,
                            linear3, relu, dropout,
                            linear4, relu, dropout,
                            linear5).to(device)

비용 함수와 옵타마이저를 선언합니다.

In [ ]:
# define cost/loss & optimizer
criterion = nn.CrossEntropyLoss().to(device) # CrossEntropyLoss()를 사용합니다.
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1) # Adam optimizer를 사용하고 위에서 정의한 learning rate를 사용합니다.

이제 모델을 훈련시켜보겠습니다.

In [ ]:
total_batch = len(data_loader)
model.train()    # set the model to train mode (dropout=True)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.view(-1, 28 * 28).to(device) # view를 이용해 X를 (100, 784)의 텐서로 변환해주세요.
        Y = Y.to(device) # 레이블은 원-핫 인코딩이 된 상태가 아니라 0 ~ 9의 정수.

        optimizer.zero_grad() # gradient를 0으로 초기화
        hypothesis = linear(X) #모델을 생성해서 hypothesis를 구해보세요.
        cost = criterion(hypothesis, Y) #위에서 정의한 비용함수를 사용하세요.
        cost.backward()# 비용 함수를 미분하여 gradient 계산
        optimizer.step()  # W와 b를 업데이트

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 0.306947082
Epoch: 0002 cost = 0.143526763
Epoch: 0003 cost = 0.110823721
Epoch: 0004 cost = 0.096670046
Epoch: 0005 cost = 0.081182100
Epoch: 0006 cost = 0.076867908
Epoch: 0007 cost = 0.067622609
Epoch: 0008 cost = 0.062523372
Epoch: 0009 cost = 0.058608234
Epoch: 0010 cost = 0.056506939
Epoch: 0011 cost = 0.049783912
Epoch: 0012 cost = 0.050642610
Epoch: 0013 cost = 0.047746319
Epoch: 0014 cost = 0.043047331
Epoch: 0015 cost = 0.044674862
Learning finished


마지막으로 모델을 테스트 해보겠습니다.

In [ ]:
# Test model and check accuracy
with torch.no_grad():
    model.eval()    # set the model to evaluation mode (dropout=False)

    # Test the model using test sets
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.979699969291687
Label:  8
Prediction:  8


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Q1. 신경망에서 레이어가 깊어질수록 어떤 문제가 일어나는가?

=> 기울기가 점차 작아지는 기울기 소실 또는 기울기가 점차 커지는 기울기 폭주의 문제   가 생길 수 있다.

Q2. 위 문제를 해결할 수 있는 방법에는 어떤 것이 있는가?

=> 1. 은닉층의 활성화 함수로 시그모이드나 tanh 함수 대신에 ReLU / ReLU의 변형 함수      와 같은 Leaky ReLU를 사용한다.
   2. 가중치가 초기에 어떤 값을 가졌느냐에 따라서 모델의 훈련 결과가 달라질 수도        있기 때문에 세이비어 초기화, He 초기화와 같은 방법으로 가중치 초기화를 해준        다.
   3. 배치정규화를 통해서 인공 신경망의 각 층에 들어가는 입력을 평균과 분산으로 정      규화하여 학습을 효율적으로 만들어줄 수 있다.
   4. 층정규화

Q3. 모델을 학습시킨 후 평가를 할 때도 Dropout을 사용하는가? (O/X)

=> X. 드롭아웃은 학습 시에 과적합을 방지하는 방법이며 학습 후에는 사용하지 않는다.